# use trait image as extent and create extent shp file

In [1]:
from osgeo import gdal,ogr,osr

# Input raster file
raster_file = r'F:\wenqu\trait_map\trait_proj\site2c_d13c_trait_project_to_aviris.tif'

# Output polygon file
polygon_file = r'F:\wenqu\Aviris\site2c\site2c_aviris_clip\site2c_d13c_trait_project_to_aviris.shp'

In [2]:
# Open the input raster file
raster_ds = gdal.Open(raster_file)

# Get the raster band and its data type
band = raster_ds.GetRasterBand(1)
data_type = band.DataType

# Get the raster's geotransform and projection
geotransform = raster_ds.GetGeoTransform()
projection = raster_ds.GetProjection()
# Create a memory layer to hold the polygon features
driver = ogr.GetDriverByName("Memory")
mem_ds = driver.CreateDataSource("")

# Create a new layer in the memory dataset
mem_layer = mem_ds.CreateLayer("polygon", srs=osr.SpatialReference(wkt=projection), geom_type=ogr.wkbPolygon)

# Use gdal.Polygonize() to convert the raster to polygons
gdal.Polygonize(band, None, mem_layer, 0, [], callback=None)
# Create the output shapefile
driver = ogr.GetDriverByName("ESRI Shapefile")
out_ds = driver.CreateDataSource(polygon_file)

# Copy the memory layer to the output shapefile
out_layer = out_ds.CopyLayer(mem_layer, "polygon")
# Clean up the memory layer
mem_ds.Destroy()

# Close the input and output files
raster_ds = None
out_ds = None

# use the created shp file to clip the aviris brdf image directly

In [1]:
import rasterio
from rasterio.mask import mask
from rasterio import Affine
import geopandas as gpd
from shapely.geometry import box
import rasterio.plot as rplot
import matplotlib.pyplot as plt
import fiona
from shapely.geometry import box
from fiona.crs import from_epsg

In [4]:
raster1 = rasterio.open(r'F:\wenqu\Aviris\site2a\output\ang20170709t012605_corr_v2p9_img_brdf')

shapefile = gpd.read_file( r'F:\wenqu\Aviris\site2c\site2c_shp\site2c_d13c_trait_project_to_aviris.shp')
geo = shapefile.geometry




# Clip raster1 using the extent of raster2:
out_img, out_transform = mask(raster1, shapes=geo.geometry, crop=True)

# Update the metadata of the output image:
out_meta = raster1.meta.copy()
out_meta.update({
    "driver": "ENVI",
    "height": out_img.shape[1],
    "width": out_img.shape[2],
    "count":425,
    "transform": out_transform
})


output = "F:/wenqu/Aviris/site2c/site2c_aviris_clip/site2c_aviris"
with rasterio.open(output, "w", **out_meta) as dst:
    # loop over the bands and write the pixel value for each band
    for band in range(1, raster1.count + 1):
        dst.write(out_img[band-1], indexes=band)

## create shp file for each aviris pixel

In [2]:
import os
from osgeo import gdal,ogr,osr
folder_path = r"F:\wenqu\Aviris\site2c\site2c_aviris_single_clip"
polygo_folder = r'F:\wenqu\Aviris\site2c\site2c_aviris_single_shp'
extension = ".tif"

for filename in os.listdir(folder_path):
    # Check if the current item is a file
    if filename.endswith(extension) and os.path.isfile(os.path.join(folder_path, filename)):
        # Do something with the file, e.g. print its name
        shp_name = os.path.splitext(filename)[0] + '.shp'
        raster_file = os.path.join(folder_path, filename)
        polygon_file = os.path.join(polygo_folder, shp_name)
        
        # Open the input raster file
        raster_ds = gdal.Open(raster_file)

        # Get the raster band and its data type
        band = raster_ds.GetRasterBand(1)
        data_type = band.DataType

        # Get the raster's geotransform and projection
        geotransform = raster_ds.GetGeoTransform()
        projection = raster_ds.GetProjection()
        # Create a memory layer to hold the polygon features
        driver = ogr.GetDriverByName("Memory")
        mem_ds = driver.CreateDataSource("")

        # Create a new layer in the memory dataset
        mem_layer = mem_ds.CreateLayer("polygon", srs=osr.SpatialReference(wkt=projection), geom_type=ogr.wkbPolygon)

        # Use gdal.Polygonize() to convert the raster to polygons
        gdal.Polygonize(band, None, mem_layer, 0, [], callback=None)
        # Create the output shapefile
        driver = ogr.GetDriverByName("ESRI Shapefile")
        out_ds = driver.CreateDataSource(polygon_file)

        # Copy the memory layer to the output shapefile
        out_layer = out_ds.CopyLayer(mem_layer, "polygon")
        # Clean up the memory layer
        mem_ds.Destroy()

        # Close the input and output files
        raster_ds = None
        out_ds = None
